# HuggingFace Transformers
While spaCy provides good-enough NER cabilitities, the accelerated pace of NLP models in recent years means that we can use pre-trained models that leverage modern machine approaches. [HuggingFace](https://huggingface.co/), a company specializing in open-source models, provides an easy-to-use Python library for applying these models on text contained in Sinopia's RDF.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext lab_black

import kglab
import pandas as pd

import helpers
import widgets

## Loading Sinopia Stage RDF Text DataFrame

In [2]:
stage_kg = kglab.KnowledgeGraph()
stage_kg.load_jsonld("data/stage.json")

http://desktop.loc.gov/search?view=document&id=Infobasedcrmg0Dash0Dash0Dash247&hl=true&fq=allresources|true# does not look like a valid URI, trying to serialize this will break.
https://api.stage.sinopia.io/resource/this is a test does not look like a valid URI, trying to serialize this will break.
ld4p:RT:bf2:2D graphic material:Item does not look like a valid URI, trying to serialize this will break.
urn:ld4p:qa:gettyaat:Objects__Object_Groupings and Systems does not look like a valid URI, trying to serialize this will break.


## Exercises